In [1]:
print('Shree Ganeshay Namah')

Shree Ganeshay Namah


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [3]:
df = pd.read_csv('Churn_Modelling.csv')
print(df.shape)
df.head()

(10000, 14)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [5]:
# drop irrelevant columns
df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
df['Gender'].value_counts()

Gender
Male      5457
Female    4543
Name: count, dtype: int64

In [7]:
# Encode categorical variable
label_enocoder_gender = LabelEncoder()
df.loc[:, 'Gender'] = label_enocoder_gender.fit_transform(df['Gender']) # Female:0 Male:1
df['Gender'].value_counts()

Gender
1    5457
0    4543
Name: count, dtype: int64

In [8]:
# One Hot Encode Geography
from sklearn.preprocessing import OneHotEncoder
ohe_geo = OneHotEncoder(sparse_output=False)
encoded_geography = ohe_geo.fit_transform(df[['Geography']])

geo_cols = ohe_geo.get_feature_names_out(['Geography'])
print(geo_cols)

df_geo = pd.DataFrame(encoded_geography, columns=geo_cols)
df_geo

['Geography_France' 'Geography_Germany' 'Geography_Spain']


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [9]:
# Combine the df with df_geo

df = pd.concat([df.drop(columns=['Geography']), df_geo], axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
# Save the encoders and scalars

with open('label_enocoder_gender.pkl', 'wb') as file:
    pickle.dump(label_enocoder_gender, file)

with open('onehot_enocoder_geography.pkl', 'wb') as file:
    pickle.dump(ohe_geo, file)

In [11]:
# Split the dat into inependent and dpendent feature

X = df.drop(columns=['Exited'], axis=1)
y = df['Exited']

# Split the data into Train and Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8000, 12), (2000, 12), (8000,), (2000,))

In [12]:
# Scale these features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled.shape, X_test.shape

((8000, 12), (2000, 12))

In [13]:
with open('standard_scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## ANN Implementation

1. ANN is sequentail network (X_train.shape)
2. Hidden Neuron: Dense from Keras with 64 nodes so 64 neuron in Hidden layer
3. Activation Function for each node such as  sigmoid, tanh, relu ,leaky relu etc...
4. Optimizer: Used during BPP to update the weights
5. Loss functions: 
6. Accuracy Metrics
7. Store Training to Logs folder , to use in Tensorboard to visualize


In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [15]:
# Build ANN
ann_model = Sequential(
    [
        Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1], )), # First hidden layer connected with I/P layer
        Dense(32, activation='relu'), # Second hidden layer
        Dense(1, activation='sigmoid'), # Output layer
    ]
)

In [16]:
ann_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
adam_aptimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

# compile the model
ann_model.compile(optimizer=adam_aptimizer, 
                  loss='binary_crossentropy',
                  metrics=['accuracy']) # for multiclass: sparse_crossentropy

In [18]:
# Set up the tensorboard to capture the logs and visualize these logs
log_dir = 'logs/fit' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [19]:
# Set up early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

In [20]:
# Train the model
history = ann_model.fit(X_train_scaled, y_train,
                        validation_data=(X_test_scaled, y_test),
                        epochs=100,
                        callbacks=[tensorflow_callback, early_stopping_callback]
                        )

Epoch 1/100


250/250 [==============================] - 1s 3ms/step - loss: 0.3983 - accuracy: 0.8359 - val_loss: 0.3497 - val_accuracy: 0.8540
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3546 - accuracy: 0.8539 - val_loss: 0.3558 - val_accuracy: 0.8545
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3491 - accuracy: 0.8550 - val_loss: 0.3797 - val_accuracy: 0.8560
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3455 - accuracy: 0.8590 - val_loss: 0.3408 - val_accuracy: 0.8600
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3416 - accuracy: 0.8610 - val_loss: 0.3439 - val_accuracy: 0.8590
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3392 - accuracy: 0.8599 - val_loss: 0.3495 - val_accuracy: 0.8570
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3382 - accuracy: 0.8600 - val_loss: 0.3457 - val_accuracy: 0.86

In [21]:
ann_model.save('ann_model.h5')

c:\Users\saura\Desktop\ws\ai\ANN-Churn-Classifier\venv3_11\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [3]:
# Load tensorboard extension
%load_ext tensorboard

In [4]:
%tensorboard --logdir logs/fit20241012-134142

Reusing TensorBoard on port 6006 (pid 3508), started 0:19:44 ago. (Use '!kill 3508' to kill it.)

#### next predictions.ipynb